Maastricht_University_logo.svg

# Information Retrieval and Text Mining Course
## Tutorial 03 — Search Engines: Relevance Ranking

**Author:** Jan Scholtes

**Edition 2025-2026**

Department of Advanced Computer Sciences — Maastricht University

Welcome to Tutorial 03 on **Search Engines: Relevance Ranking**. In this tutorial we explore how search engines rank documents by relevance, progressing from classical lexical methods to neural approaches.

The tutorial is organised in three stages:

1. **Stage 1 — BM25 Baseline**: We use [Pyserini](https://github.com/castorini/pyserini) to perform BM25 retrieval on the MS MARCO passage corpus with official TREC Deep Learning 2019 queries. We observe where keyword-based ranking succeeds and where it fails.
2. **Stage 2 — Neural Reranking**: We apply a neural cross-encoder to rerank the BM25 results, demonstrating how semantic understanding produces better rankings.
3. **Stage 3 — Quantitative Evaluation**: We compute nDCG@10 and MAP using official NIST relevance judgments (qrels) to show rigorous evidence that neural reranking outperforms BM25.

Before the experiment we review the theory behind TF-IDF, BM25, and neural ranking methods.

**Dataset**: MS MARCO Passage Ranking with TREC Deep Learning 2019 evaluation queries (43 queries with graded relevance judgments from NIST assessors).

> **Note:** This course is about Information Retrieval, Text Mining, and Conversational Search — not about programming skills. The code cells below show you *how* these methods work in practice using Python libraries. Focus on understanding the **concepts** and **results**.

## Library Installation

We install all required packages in a single cell. Run this cell once at the beginning of your session.

**Important:** Pyserini requires **Java 11+** (JDK, not just JRE). If you do not have Java installed:
- **Windows:** `winget install Microsoft.OpenJDK.21`
- **macOS:** `brew install openjdk@21`
- **Linux:** `sudo apt install openjdk-21-jdk`

The first time you run the search cells, Pyserini will download the MS MARCO passage index (~2 GB). This is a one-time operation.

In [ ]:
# Install required packages
import subprocess, sys

packages = [
    "pyserini",
    "sentence-transformers",
    "faiss-cpu",
    "scikit-learn",
]
for pkg in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print("All packages installed successfully.")

In [ ]:
# --- Java & Environment Setup ---
import os, json, math, sys, warnings
import numpy as np
from collections import Counter, defaultdict

warnings.filterwarnings("ignore")

# Pyserini requires JAVA_HOME pointing to a JDK 11+ installation.
# The cell below tries to auto-detect your JDK. If it fails, set the
# path manually: os.environ["JAVA_HOME"] = r"C:\path\to\jdk"
if "JAVA_HOME" not in os.environ or not os.environ["JAVA_HOME"]:
    import glob
    candidates = (
        glob.glob(r"C:\Program Files\Microsoft\jdk-*")
        + glob.glob(r"C:\Program Files\Java\jdk-*")
        + glob.glob("/usr/lib/jvm/java-*-openjdk*")
        + glob.glob("/Library/Java/JavaVirtualMachines/*/Contents/Home")
    )
    if candidates:
        os.environ["JAVA_HOME"] = sorted(candidates)[-1]
        print(f"Auto-detected JAVA_HOME: {os.environ['JAVA_HOME']}")
    else:
        raise EnvironmentError(
            "JAVA_HOME not set and no JDK found.\n"
            "Install JDK 11+ first (e.g. winget install Microsoft.OpenJDK.21)"
        )

# --- Pyserini & model imports ---
from pyserini.search.lucene import LuceneSearcher
from pyserini.search import get_topics, get_qrels
from sentence_transformers import CrossEncoder
import torch

print(f"Pyserini loaded  | JAVA_HOME: {os.environ['JAVA_HOME']}")
print(f"PyTorch {torch.__version__} | CUDA available: {torch.cuda.is_available()}")

## 1. From Keywords to Meaning: The Relevance Problem

At the heart of every search engine is a **ranking function** — a mathematical formula that decides which documents are most relevant to a query.

The simplest approach is **exact keyword matching**: find documents that contain the query terms and rank them by how often those terms appear. This works surprisingly well, but it has a fundamental limitation called the **lexical gap**:

> A user searching for *"how to fix a broken screen"* may not find a document titled *"smartphone display repair guide"* because the words do not match — even though the meaning is the same.

This tutorial explores the evolution from keyword-based to semantic ranking:

| Generation | Method | Matching | Limitation |
|-----------|--------|----------|------------|
| 1st | TF-IDF | Exact term overlap | No term importance model |
| 2nd | BM25 | Probabilistic term weighting | Still requires word overlap |
| 3rd | Neural (BERT, ColBERT) | Semantic similarity | Computationally expensive |

We will see this progression **in practice** using a real search engine (Pyserini) and a real evaluation benchmark (TREC-DL 2019).

## 2. TF-IDF: The Foundation of Lexical Ranking

**Term Frequency – Inverse Document Frequency** (TF-IDF) is the most widely used term-weighting scheme.

### Term Frequency (TF)

The raw term frequency $f(t, d)$ counts how often term $t$ appears in document $d$. To dampen the effect of very frequent terms we often use the log variant:

$$\text{TF}(t, d) = 1 + \log f(t, d) \quad\text{if } f(t, d) > 0,\;\text{else } 0$$

### Inverse Document Frequency (IDF)

A term that appears in many documents is less informative. IDF captures this:

$$\text{IDF}(t) = \log \frac{N}{df(t)}$$

where $N$ is the total number of documents and $df(t)$ is the number of documents containing term $t$.

### Combined TF-IDF Score

$$\text{TF\text{-}IDF}(t, d) = \text{TF}(t, d) \times \text{IDF}(t)$$

The score for a query $q$ against document $d$ sums over all query terms:

$$\text{Score}(q, d) = \sum_{t \in q} \text{TF\text{-}IDF}(t, d)$$

In [ ]:
# TF-IDF demonstration on toy documents

documents = [
    "information retrieval is the science of searching for information",
    "machine learning models can improve search relevance",
    "information retrieval systems use inverted indexes for fast search",
    "deep learning transforms natural language understanding",
]
query = "information retrieval search"

# Tokenise
def tokenize(text):
    return text.lower().split()

doc_tokens = [tokenize(d) for d in documents]
query_tokens = tokenize(query)
N = len(documents)

# Compute document frequency & IDF
df = Counter()
for tokens in doc_tokens:
    for t in set(tokens):
        df[t] += 1

idf = {t: math.log(N / df[t]) for t in df}

# Score each document
print(f"Query: '{query}'\n")
print(f"{'Term':<20} {'IDF':>6}")
print("-" * 28)
for t in query_tokens:
    print(f"{t:<20} {idf.get(t, 0):>6.3f}")

print(f"\n{'Doc':<5} {'Score':>8}  Content")
print("-" * 75)
for i, tokens in enumerate(doc_tokens):
    tf = Counter(tokens)
    score = sum(
        (1 + math.log(tf[t])) * idf.get(t, 0)
        for t in query_tokens if tf[t] > 0
    )
    print(f"D{i:<4} {score:>8.3f}  {documents[i]}")

## 3. BM25: The Best Lexical Ranker

**BM25** (Best Match 25) is a probabilistic ranking function developed at City University London in the 1990s as part of the Okapi system. It extends TF-IDF with two important improvements:

1. **Saturation** — term frequency has diminishing returns (a word appearing 100 times is not 100× more relevant than appearing once)
2. **Length normalisation** — longer documents are not automatically favoured

### The BM25 Formula

$$\text{BM25}(q, d) = \sum_{t \in q} \text{IDF}(t) \;\cdot\; \frac{f(t,d) \cdot (k_1 + 1)}{f(t,d) + k_1 \cdot \left(1 - b + b \cdot \dfrac{|d|}{\text{avgdl}}\right)}$$

where:
- $f(t, d)$ = frequency of term $t$ in document $d$
- $|d|$ = length of document $d$ (in words)
- $\text{avgdl}$ = average document length in the collection
- $k_1$ = term frequency saturation parameter (typically **1.2**)
- $b$ = length normalisation parameter (typically **0.75**)

### IDF Component

$$\text{IDF}(t) = \log \frac{N - df(t) + 0.5}{df(t) + 0.5}$$

### Understanding the Parameters

| $k_1$ | Effect |
|-------|--------|
| $\to 0$ | All non-zero term frequencies treated equally (binary matching) |
| $\to \infty$ | Raw term frequency dominates (no saturation) |

| $b$ | Effect |
|-----|--------|
| $= 0$ | No length normalisation |
| $= 1$ | Full normalisation relative to average length |

BM25 remains the **default baseline** in modern information retrieval and is the first-stage retriever in most production search systems.

In [ ]:
# Explore how BM25 parameters k1 and b affect scoring

def bm25_term_weight(tf, dl, avgdl, N, df, k1=1.2, b=0.75):
    """BM25 weight for a single term in a document."""
    idf = math.log((N - df + 0.5) / (df + 0.5))
    tf_part = (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * dl / avgdl))
    return idf * tf_part

# Fixed parameters (MS MARCO scale)
N, df_val, avgdl = 8_800_000, 50_000, 60

print("Effect of k1 (term frequency saturation)  [df=50 000, avgdl=60, dl=60]")
print(f"{'TF':>4}", end="")
for k1 in [0.01, 0.5, 1.2, 3.0, 10.0]:
    print(f"  k1={k1:<5}", end="")
print()
print("-" * 55)
for tf in [1, 2, 5, 10, 20, 50]:
    print(f"{tf:>4}", end="")
    for k1 in [0.01, 0.5, 1.2, 3.0, 10.0]:
        w = bm25_term_weight(tf, 60, avgdl, N, df_val, k1=k1, b=0.75)
        print(f"  {w:>8.2f}", end="")
    print()

print(f"\nEffect of b (length normalisation)  [tf=3, k1=1.2, avgdl={avgdl}]")
print(f"{'DocLen':>7}", end="")
for b in [0.0, 0.25, 0.5, 0.75, 1.0]:
    print(f"   b={b:<5}", end="")
print()
print("-" * 55)
for dl in [20, 40, 60, 100, 200, 500]:
    print(f"{dl:>7}", end="")
    for b in [0.0, 0.25, 0.5, 0.75, 1.0]:
        w = bm25_term_weight(3, dl, avgdl, N, df_val, k1=1.2, b=b)
        print(f"  {w:>8.2f}", end="")
    print()

print("\nKey observations:")
print("  Higher k1  ->  less saturation  ->  raw TF matters more")
print("  Higher b   ->  more length normalisation  ->  short documents boosted")
print("  Default (k1=1.2, b=0.75) balances both effects")

## 4. Stage 1 — BM25 Search with Pyserini

Now we move from theory to practice. We use [Pyserini](https://github.com/castorini/pyserini), a Python toolkit for reproducible IR research, to perform BM25 search on a real corpus.

### Dataset: MS MARCO Passages + TREC-DL 2019

| Component | Description |
|-----------|-------------|
| **Corpus** | MS MARCO v1 passage collection — **8.8 million passages** from web documents (Microsoft) |
| **Queries** | 43 queries from TREC Deep Learning 2019, selected by NIST for rigorous evaluation |
| **Relevance judgments** | Graded assessments by NIST assessors (0 = not relevant … 3 = perfectly relevant) |

Pyserini provides a **pre-built Lucene index** for MS MARCO, so we can start searching immediately.

> **Note:** The first run downloads the pre-built index (~2 GB). Subsequent runs use the cached version.

In [ ]:
# Load pre-built index, queries, and relevance judgments
print("Loading MS MARCO passage index (first run downloads ~2 GB)...")
searcher = LuceneSearcher.from_prebuilt_index('msmarco-v1-passage')
print(f"Index loaded: {searcher.num_docs:,} passages")

# TREC Deep Learning 2019 topics and official qrels
topics = get_topics('dl19-passage')
qrels  = get_qrels('dl19-passage')
print(f"TREC-DL 2019 queries: {len(topics)}")
print(f"TREC-DL 2019 qrels  : {len(qrels)} queries with judgments")

# Show a few example queries
print("\nExample queries:")
for i, (qid, topic) in enumerate(topics.items()):
    if i >= 6:
        break
    n_rel = sum(1 for r in qrels.get(qid, {}).values() if r >= 2)
    print(f"  [{qid}] {topic['title']:<55} ({n_rel} highly-relevant docs)")

In [ ]:
# Run BM25 search for all TREC-DL 2019 queries (top-100 per query)
TOP_K = 100
bm25_results = {}   # {qid: [(docid, bm25_score, passage_text), ...]}

print(f"Running BM25 search (top-{TOP_K}) for {len(topics)} queries...")
for qid, topic in topics.items():
    query = topic['title']
    hits = searcher.search(query, k=TOP_K)
    results = []
    for hit in hits:
        doc = searcher.doc(hit.docid)
        passage = json.loads(doc.raw())['contents']
        results.append((hit.docid, hit.score, passage))
    bm25_results[qid] = results

print(f"BM25 search complete: {len(bm25_results)} queries processed")
print(f"Average results per query: "
      f"{np.mean([len(v) for v in bm25_results.values()]):.0f}")

In [ ]:
# Display BM25 top-5 for a few example queries
example_qids = list(topics.keys())[:3]

for qid in example_qids:
    query = topics[qid]['title']
    results = bm25_results[qid]
    print(f"\n{'='*80}")
    print(f"Query [{qid}]: {query}")
    print(f"{'='*80}")
    for rank, (docid, score, passage) in enumerate(results[:5], 1):
        rel = qrels.get(qid, {}).get(docid, '-')
        print(f"\n  Rank {rank} | BM25: {score:.4f} | Rel: {rel} | {docid}")
        print(f"  {passage[:150]}...")

### The Lexical Gap

BM25 works well when query and document share the same vocabulary. But what happens when they use **different words for the same concept**?

| Query | Relevant passage uses… | BM25 can match? |
|-------|------------------------|-----------------|
| "fix broken screen" | "display repair guide" | No word overlap |
| "heart attack symptoms" | "signs of myocardial infarction" | Medical synonyms |
| "affordable housing" | "low-cost residential options" | Paraphrases |

This is the **vocabulary mismatch problem** — the fundamental limitation of all lexical methods, including BM25. No matter how sophisticated the term weighting, if the words do not match the document will not be found.

Let us quantify this on our TREC-DL data: how many highly-relevant documents does BM25 actually find?

In [ ]:
# Lexical Gap Analysis: how many highly-relevant docs (qrel >= 2)
# appear in BM25 top-100?

print("Highly-relevant documents (qrel >= 2) found in BM25 top-100\n")
print(f"{'QID':<10} {'Query':<45} {'Found':>5} {'Total':>5} {'Recall':>7}")
print("-" * 78)

recall_values = []
for qid in sorted(topics.keys()):
    if qid not in qrels:
        continue
    query = topics[qid]['title']
    relevant = {did for did, r in qrels[qid].items() if r >= 2}
    if not relevant:
        continue
    retrieved = {docid for docid, _, _ in bm25_results.get(qid, [])}
    found = relevant & retrieved
    recall = len(found) / len(relevant)
    recall_values.append(recall)
    print(f"{qid:<10} {query[:43]:<45} {len(found):>5} {len(relevant):>5} {recall:>7.1%}")

print(f"\nMean recall of highly-relevant docs: {np.mean(recall_values):.1%}")
print(f"Queries with < 100% recall: "
      f"{sum(1 for r in recall_values if r < 1.0)}/{len(recall_values)}")
print("\nBM25 misses some relevant passages — this is the lexical gap in action.")

## 5. Neural Ranking: Beyond Exact Match

Neural ranking models use **learned representations** (embeddings) to capture semantic similarity between queries and documents. Instead of matching words they match *meanings*.

### Three Architectures for Neural Ranking

| Architecture | Example | How it works | Speed | Quality |
|-------------|---------|-------------|-------|---------|
| **Bi-encoder** | DPR, SBERT | Query and doc encoded independently; cosine similarity | Fast | Good |
| **Late interaction** | ColBERT | Token-level embeddings; MaxSim aggregation | Medium | Better |
| **Cross-encoder** | monoBERT | Query + doc processed jointly by a transformer | Slow | Best |

### ColBERT: Late Interaction via MaxSim

ColBERT (Contextualized Late Interaction over BERT) scores a query–document pair by:

1. Encoding query tokens:  $\mathbf{Q} = [\mathbf{q}_1, \mathbf{q}_2, \ldots, \mathbf{q}_m]$
2. Encoding document tokens:  $\mathbf{D} = [\mathbf{d}_1, \mathbf{d}_2, \ldots, \mathbf{d}_n]$
3. Computing **MaxSim** — for each query token, find its maximum similarity to any document token:

$$\text{ColBERT}(q, d) = \sum_{i=1}^{m} \max_{j=1}^{n}\; \mathbf{q}_i^\top \mathbf{d}_j$$

This captures fine-grained token-level semantic matches — for example "screen" matching "display" through contextual embeddings.

### Cross-Encoder: Joint Encoding

A cross-encoder feeds the concatenated query–document pair through BERT:

$$\text{Score}(q, d) = \text{BERT}_{\text{cls}}\!\bigl([\texttt{CLS}]\; q \;[\texttt{SEP}]\; d \;[\texttt{SEP}]\bigr)$$

Cross-encoders are the most powerful but slowest neural rankers — they are used to **rerank** a small candidate set retrieved by BM25.

In this tutorial we use a cross-encoder to rerank BM25's top-100 results. This demonstrates the same principle as ColBERT: **semantic understanding beats keyword matching**.

## 6. Stage 2 — Neural Reranking of BM25 Results

We now apply a neural cross-encoder to rerank the BM25 top-100 results. The model — `cross-encoder/ms-marco-MiniLM-L-6-v2` — was trained on the MS MARCO dataset to predict query–passage relevance.

**Pipeline:**
1. **BM25** retrieves top-100 candidate passages (fast, recall-oriented)
2. **Cross-encoder** scores each (query, passage) pair (slower, precision-oriented)
3. Passages are **reranked** by the cross-encoder score

This **retrieve-then-rerank** pattern is the standard approach in modern search engines.

In [ ]:
# Load cross-encoder model
print("Loading cross-encoder: cross-encoder/ms-marco-MiniLM-L-6-v2 ...")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
print("Cross-encoder loaded.\n")

# Rerank all queries
reranked_results = {}   # {qid: [(docid, ce_score, passage), ...]}

print(f"Reranking {len(bm25_results)} queries (top-{TOP_K} each)...")
for i, (qid, results) in enumerate(bm25_results.items()):
    query = topics[qid]['title']

    # Prepare (query, passage) pairs
    pairs = [(query, passage) for _, _, passage in results]

    # Score all pairs at once
    ce_scores = cross_encoder.predict(pairs, show_progress_bar=False)

    # Sort by cross-encoder score (descending)
    reranked = sorted(
        [(docid, float(sc), passage)
         for (docid, _, passage), sc in zip(results, ce_scores)],
        key=lambda x: x[1],
        reverse=True,
    )
    reranked_results[qid] = reranked

    if (i + 1) % 10 == 0 or (i + 1) == len(bm25_results):
        print(f"  {i+1}/{len(bm25_results)} queries reranked")

print("Neural reranking complete.")

In [ ]:
# Side-by-side: BM25 ranking vs Neural reranking (top-10)
example_qids = list(topics.keys())[:3]

for qid in example_qids:
    query = topics[qid]['title']

    # Build BM25 rank lookup
    bm25_rank = {docid: r for r, (docid, _, _)
                 in enumerate(bm25_results[qid], 1)}

    print(f"\n{'='*90}")
    print(f"Query [{qid}]: {query}")
    print(f"{'='*90}")
    print(f"  {'#':>3} {'BM25#':>6} {'Move':>6} {'CE Score':>9} {'Rel':>4}  Passage")
    print(f"  {'-'*82}")

    for rank, (docid, ce_score, passage) in enumerate(reranked_results[qid][:10], 1):
        old = bm25_rank.get(docid, TOP_K + 1)
        delta = old - rank
        if delta > 0:
            arrow = f"+{delta}"
        elif delta < 0:
            arrow = str(delta)
        else:
            arrow = "="
        rel = qrels.get(qid, {}).get(docid, '-')
        print(f"  {rank:>3} {old:>6} {arrow:>6} {ce_score:>9.4f} {str(rel):>4}"
              f"  {passage[:50]}...")

In [ ]:
# Aggregate rank-change analysis
total_up, total_big, total_pairs = 0, 0, 0

for qid in bm25_results:
    bm25_rank = {did: r for r, (did, _, _) in enumerate(bm25_results[qid], 1)}
    for new_rank, (docid, _, _) in enumerate(reranked_results[qid][:10], 1):
        old_rank = bm25_rank.get(docid, TOP_K + 1)
        change = old_rank - new_rank
        if change > 0:
            total_up += 1
        if change >= 10:
            total_big += 1
        total_pairs += 1

print("Rank-Change Analysis  (Neural top-10 vs BM25 ranking)")
print("-" * 50)
print(f"Total (query, passage) pairs : {total_pairs}")
print(f"Passages moved UP            : {total_up}  "
      f"({100*total_up/total_pairs:.1f}%)")
print(f"Big jumps (moved up >= 10)   : {total_big}  "
      f"({100*total_big/total_pairs:.1f}%)")
print()
print("Neural reranking reshuffles the top results significantly,")
print("promoting semantically relevant passages that BM25 ranked lower.")

## 7. Evaluation Metrics: nDCG and MAP

How do we **objectively measure** whether one ranking is better than another? We use standard IR evaluation metrics computed against human relevance judgments.

### Normalised Discounted Cumulative Gain (nDCG@k)

nDCG rewards relevant documents at high positions using **graded relevance** (0, 1, 2, 3):

$$\text{DCG}@k = \sum_{i=1}^{k} \frac{2^{rel_i} - 1}{\log_2(i + 1)}$$

$$\text{nDCG}@k = \frac{\text{DCG}@k}{\text{IDCG}@k}$$

where IDCG is the DCG of the ideal (perfect) ranking. nDCG ranges from 0 to 1.

### Mean Average Precision (MAP)

MAP measures how well **all** relevant documents are ranked, using **binary relevance**:

$$\text{AP}(q) = \frac{1}{|R_q|} \sum_{k=1}^{n} P@k \cdot \text{rel}(k)$$

$$\text{MAP} = \frac{1}{|Q|} \sum_{q \in Q} \text{AP}(q)$$

where $P@k$ is precision at rank $k$ and $\text{rel}(k)$ is 1 if the document at rank $k$ is relevant.

In [ ]:
# Implementation of evaluation metrics

def dcg_at_k(relevances, k):
    """DCG@k given a list of relevance scores in ranking order."""
    return sum(
        (2 ** rel - 1) / math.log2(i + 2)
        for i, rel in enumerate(relevances[:k])
    )

def ndcg_at_k(ranked_rels, all_rels, k):
    """nDCG@k: ranked_rels = relevances in system order;
    all_rels = all known relevance values (for IDCG)."""
    idcg = dcg_at_k(sorted(all_rels, reverse=True), k)
    if idcg == 0:
        return 0.0
    return dcg_at_k(ranked_rels, k) / idcg

def average_precision(ranked_binary, total_relevant):
    """Average Precision given binary relevances in ranking order."""
    if total_relevant == 0:
        return 0.0
    ap, hits = 0.0, 0
    for i, rel in enumerate(ranked_binary):
        if rel:
            hits += 1
            ap += hits / (i + 1)
    return ap / total_relevant

# Sanity check
test_rels = [3, 2, 0, 1, 0, 0, 2, 0, 0, 0]
print("Sanity check — test ranking:", test_rels)
print(f"  DCG@10 : {dcg_at_k(test_rels, 10):.4f}")
print(f"  nDCG@10: {ndcg_at_k(test_rels, test_rels, 10):.4f}")
test_bin = [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
print(f"  AP     : {average_precision(test_bin, 4):.4f}")

## 8. Stage 3 — Quantitative Comparison

We now evaluate both ranking approaches — **BM25** and **Neural Reranking** — using the official TREC-DL 2019 relevance judgments.

For each of the 43 queries we compute:
- **nDCG@10**: graded relevance quality at the top of the ranking
- **MAP** (with relevance threshold $\geq 2$): binary precision across all ranks

In [ ]:
# Evaluate BM25 vs Neural Reranking on TREC-DL 2019
K = 10

bm25_ndcg_list, neural_ndcg_list = [], []
bm25_ap_list, neural_ap_list     = [], []

print(f"{'QID':<10} {'Query':<40} {'BM25':>8} {'Neural':>8} {'Delta':>8}")
print("-" * 78)

for qid in sorted(topics.keys()):
    if qid not in qrels or qid not in bm25_results:
        continue

    q_qrels = qrels[qid]
    query   = topics[qid]['title']
    all_rels = list(q_qrels.values())

    # Relevance scores in system ranking order
    bm25_rels   = [q_qrels.get(did, 0) for did, _, _ in bm25_results[qid][:K]]
    neural_rels = [q_qrels.get(did, 0) for did, _, _ in reranked_results[qid][:K]]

    # nDCG@K
    b_ndcg = ndcg_at_k(bm25_rels, all_rels, K)
    n_ndcg = ndcg_at_k(neural_rels, all_rels, K)

    # MAP (binary: relevant if qrel >= 2)
    total_rel = sum(1 for r in q_qrels.values() if r >= 2)
    b_ap = average_precision(
        [1 if q_qrels.get(did, 0) >= 2 else 0 for did, _, _ in bm25_results[qid]],
        total_rel)
    n_ap = average_precision(
        [1 if q_qrels.get(did, 0) >= 2 else 0 for did, _, _ in reranked_results[qid]],
        total_rel)

    bm25_ndcg_list.append(b_ndcg)
    neural_ndcg_list.append(n_ndcg)
    bm25_ap_list.append(b_ap)
    neural_ap_list.append(n_ap)

    d = n_ndcg - b_ndcg
    flag = "+" if d > 0 else ("=" if d == 0 else "-")
    print(f"{qid:<10} {query[:38]:<40} {b_ndcg:>8.4f} {n_ndcg:>8.4f} {d:>+8.4f} {flag}")

In [ ]:
# Summary statistics
bm25_ndcg  = np.mean(bm25_ndcg_list)
neural_ndcg = np.mean(neural_ndcg_list)
bm25_map   = np.mean(bm25_ap_list)
neural_map  = np.mean(neural_ap_list)

pct_ndcg = (neural_ndcg - bm25_ndcg) / bm25_ndcg * 100 if bm25_ndcg else 0
pct_map  = (neural_map - bm25_map) / bm25_map * 100 if bm25_map else 0

wins   = sum(1 for b, n in zip(bm25_ndcg_list, neural_ndcg_list) if n > b)
ties   = sum(1 for b, n in zip(bm25_ndcg_list, neural_ndcg_list) if n == b)
losses = sum(1 for b, n in zip(bm25_ndcg_list, neural_ndcg_list) if n < b)

print()
print("=" * 62)
print("       TREC-DL 2019  —  BM25  vs  Neural Reranking")
print("=" * 62)
print(f"\n{'Metric':<18} {'BM25':>10} {'Neural':>10} {'Improve':>10}")
print("-" * 52)
print(f"{'nDCG@10':<18} {bm25_ndcg:>10.4f} {neural_ndcg:>10.4f} {pct_ndcg:>+9.1f}%")
print(f"{'MAP (rel>=2)':<18} {bm25_map:>10.4f} {neural_map:>10.4f} {pct_map:>+9.1f}%")
print(f"\nWin / Tie / Loss (nDCG@10):  {wins}W / {ties}T / {losses}L")
print(f"\nConclusion: Neural reranking "
      f"{'outperforms' if neural_ndcg > bm25_ndcg else 'matches'} "
      f"BM25 by {abs(pct_ndcg):.1f}% in nDCG@10 on TREC-DL 2019.")

### Discussion

The results demonstrate a clear and consistent pattern:

1. **BM25 is a strong baseline** — it achieves reasonable nDCG scores by effectively matching query terms to passage terms using probabilistic weighting.

2. **Neural reranking consistently improves over BM25** — the cross-encoder processes query and passage jointly, capturing semantic relationships that BM25 misses:
   - Synonyms ("car" ↔ "automobile")
   - Paraphrases ("how to fix" ↔ "repair instructions")
   - Conceptual similarity ("heart attack" ↔ "myocardial infarction")

3. **The retrieve-then-rerank pipeline is practical** — BM25 provides fast recall over 8.8 million passages; the neural model refines the top-100 with semantic scoring.

This is the same pattern used in production search engines and forms the basis for more advanced systems covered in later tutorials (RAG, Conversational Search).

## 9. Summary

| Method | Approach | Strength | Weakness |
|--------|----------|----------|----------|
| **TF-IDF** | Term frequency $\times$ inverse document frequency | Simple, interpretable | No saturation, no length norm |
| **BM25** | Probabilistic model with saturation ($k_1$) and length norm ($b$) | Strong baseline, fast | Lexical gap |
| **Neural (Cross-encoder)** | Transformer-based semantic scoring | Captures meaning, state-of-the-art | Slow per pair, needs BM25 first |

### Key Takeaways

1. **BM25**: $\displaystyle\sum_{t \in q} \text{IDF}(t)\;\frac{f(t,d)(k_1+1)}{f(t,d)+k_1(1-b+b\,|d|/\text{avgdl})}$ — the default first-stage retriever
2. **The lexical gap** is the fundamental limitation of all keyword-based methods
3. **ColBERT MaxSim**: $\sum_i \max_j\;\mathbf{q}_i^\top\mathbf{d}_j$ — token-level semantic scoring
4. **Retrieve-then-rerank** (BM25 → Neural) is the dominant paradigm in modern search
5. **nDCG@k** and **MAP** are the standard metrics for evaluating ranking quality
6. Neural reranking **significantly outperforms** BM25 on TREC-DL benchmarks

---

## Exercises

The following exercises are graded. You are expected to answer them on your own.

## Exercise 1 — BM25 Parameter Analysis (5 points)

The BM25 formula uses two key parameters: $k_1$ (term frequency saturation) and $b$ (document length normalisation).

1. Explain the **intuition** behind each parameter. What does each control and why is it needed?
2. If you are searching a collection of **scientific abstracts** (all approximately the same length, ~200 words), how would you adjust $b$? Justify your answer.
3. If you are searching a collection where **repeated keywords are a strong signal of relevance** (e.g., product reviews mentioning a specific feature), how would you adjust $k_1$? Justify your answer.
4. Explain why BM25 with $k_1 \to 0$ and $b = 0$ reduces to a simpler scoring model. Which model does it approximate?

Write your answer in the cell below (minimum 150 words).

BEGIN SOLUTION

END SOLUTION

YOUR ANSWER HERE

## Exercise 2 — The Lexical Gap and Neural Solutions (5 points)

Consider the following search scenario.

**Query:** *"renewable energy impact on wildlife"*

BM25 retrieves these top-3 passages:
1. *"Renewable energy sources include solar, wind, and hydroelectric power. These energy sources are considered renewable because they are naturally replenished."*
2. *"The impact of energy production on the environment has been studied extensively. Wind farms and solar panels are common renewable energy installations."*
3. *"Wildlife conservation efforts focus on protecting endangered species and their natural habitats from human activities."*

A passage rated **highly relevant** by NIST assessors but ranked at position 87 by BM25:
- *"Bird and bat mortality near wind turbines has raised ecological concerns. Studies show that migratory species are particularly vulnerable to collisions with turbine blades, highlighting the tension between clean power generation and biodiversity preservation."*

Answer the following:

1. Explain **why** BM25 ranked the relevant passage so low. Identify the specific vocabulary mismatches.
2. Explain how a **ColBERT MaxSim** model would handle this differently. Reference the specific token-level matches that MaxSim would capture.
3. Would a **bi-encoder** (like DPR) also solve this problem? How does its approach differ from ColBERT's token-level interaction?

Write your answer in the cell below (minimum 200 words).

BEGIN SOLUTION

END SOLUTION

YOUR ANSWER HERE

## Exercise 3 — Implementing Precision@k and Recall@k (10 points)

Write code that computes **Precision@k** and **Recall@k** for both BM25 and Neural reranking on the TREC-DL 2019 data used in this tutorial.

Your code should:

1. Implement `precision_at_k(ranked_docids, qrels_dict, k)` — returns the fraction of the top-*k* documents that are relevant (qrel $\geq$ 2)
2. Implement `recall_at_k(ranked_docids, qrels_dict, k)` — returns the fraction of all relevant documents (qrel $\geq$ 2) that appear in the top-*k*
3. Compute the **mean** Precision@10 and Recall@10 across all TREC-DL 2019 queries for both `bm25_results` and `reranked_results`
4. Store the results in four variables: `p10_bm25`, `p10_neural`, `r10_bm25`, `r10_neural` (all floats between 0 and 1)

**Variables available** (defined earlier in this notebook):
- `bm25_results` — dict: query ID → list of `(docid, bm25_score, passage)` tuples
- `reranked_results` — dict: query ID → list of `(docid, ce_score, passage)` tuples
- `qrels` — dict: query ID → dict `{docid: relevance_score}`
- `topics` — dict: query ID → topic dict with `'title'` key

BEGIN SOLUTION

END SOLUTION

In [ ]:
# YOUR CODE HERE
raise NotImplementedError("Replace this line with your solution")

In [ ]:
# Autograder test cell — do not modify
assert 'p10_bm25' in dir(), "Define 'p10_bm25'"
assert 'p10_neural' in dir(), "Define 'p10_neural'"
assert 'r10_bm25' in dir(), "Define 'r10_bm25'"
assert 'r10_neural' in dir(), "Define 'r10_neural'"

for name, val in [('p10_bm25', p10_bm25), ('p10_neural', p10_neural),
                   ('r10_bm25', r10_bm25), ('r10_neural', r10_neural)]:
    assert isinstance(val, float), f"{name} should be a float, got {type(val)}"
    assert 0 <= val <= 1, f"{name} should be in [0, 1], got {val}"

# Neural should outperform or match BM25 on precision
assert p10_neural >= p10_bm25 - 0.01, (
    f"Expected neural P@10 ({p10_neural:.4f}) >= BM25 P@10 ({p10_bm25:.4f})")

print(f"{'Method':<12} {'P@10':>8} {'R@10':>8}")
print("-" * 30)
print(f"{'BM25':<12} {p10_bm25:>8.4f} {r10_bm25:>8.4f}")
print(f"{'Neural':<12} {p10_neural:>8.4f} {r10_neural:>8.4f}")
print(f"\nAll auto-graded tests passed!")